In [1]:
import pandas as pd
from googleapiclient.discovery import build

def get_youtube_comments_df(video_id, api_key):
    youtube = build('youtube', 'v3', developerKey=api_key)
    comments_data = []
    
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100,
        textFormat="plainText"
    )

    while request:
        response = request.execute()
        for item in response['items']:
            snippet = item['snippet']['topLevelComment']['snippet']
            comments_data.append({
                'Author': snippet['authorDisplayName'],
                'Comment': snippet['textDisplay'],
                'Likes': snippet['likeCount'],
                'Timestamp': snippet['publishedAt'],
                'ReplyCount': item['snippet']['totalReplyCount']
            })

        if 'nextPageToken' in response:
            request = youtube.commentThreads().list_next(request, response)
        else:
            break

    # Convert list of dictionaries into a Pandas DataFrame
    df = pd.DataFrame(comments_data)
    return df

# --- EXECUTION ---
API_KEY = "[your api]"
VIDEO_ID = "OBDJgW-TJKc"

one_piece_s2_df = get_youtube_comments_df(VIDEO_ID, API_KEY)

# 1. Sort by most liked comments
one_piece_s2_df = one_piece_s2_df.sort_values(by='Likes', ascending=False)

# 2. Save to CSV
one_piece_s2_df.to_csv('one_piece_s2.csv', index=False, encoding='utf-8-sig')

print(f"Scraped {len(one_piece_s2_df)} comments. Here are the top 5:")
print(one_piece_s2_df.head())

Scraped 6703 comments. Here are the top 5:
                       Author  \
5600                 @EN-Fitz   
4127        @invisiblefly2454   
6516  @Parzival.UltimateGamer   
6228             @NhojLhiac27   
6289               @jacenwade   

                                                Comment  Likes  \
5600  Chopper’s hiding skill has obviously been impr...  18073   
4127  It's such a miracle that the show is allowed t...  16158   
6516  We better have that one crocus gag with the dr...  14293   
6228  Fully embracing and not shying away from the r...   8388   
6289  They actually adapted the unluckies, lol.\n\nG...   7077   

                 Timestamp  ReplyCount  
5600  2026-01-12T16:11:16Z          79  
4127  2026-01-12T19:58:29Z         150  
6516  2026-01-12T15:09:10Z         131  
6228  2026-01-12T15:27:06Z          42  
6289  2026-01-12T15:24:33Z          57  
